# Backtest Analysis

This notebook analyzes backtest results from the artifacts directory.

## Features:
- Equity curve visualization
- P&L distribution analysis
- Top winning and losing trades
- Performance metrics summary

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Figure size
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Load Backtest Data

Load the latest backtest artifacts from the artifacts directory.

In [ ]:
# Find latest artifacts
artifacts_dir = Path('../artifacts')

# Get the latest files
trade_files = sorted(artifacts_dir.glob('trades_*.csv'))
equity_files = sorted(artifacts_dir.glob('equity_curve_*.csv'))
summary_files = sorted(artifacts_dir.glob('summary_*.json'))

if not trade_files or not equity_files or not summary_files:
    print("No artifacts found! Please run a backtest first.")
else:
    # Load latest files
    trades_df = pd.read_csv(trade_files[-1])
    equity_df = pd.read_csv(equity_files[-1])
    
    with open(summary_files[-1], 'r') as f:
        summary = json.load(f)
    
    print(f"Loaded backtest results from:")
    print(f"  Trades: {trade_files[-1].name}")
    print(f"  Equity: {equity_files[-1].name}")
    print(f"  Summary: {summary_files[-1].name}")
    print(f"\nTotal trades: {len(trades_df)}")

## 2. Summary Statistics

In [ ]:
# Display summary metrics
metrics = summary.get('metrics', {})

print("=" * 60)
print("BACKTEST PERFORMANCE SUMMARY")
print("=" * 60)
print(f"\nProfit & Loss:")
print(f"  Total P&L: ${metrics.get('total_pnl', metrics.get('total_profit', 0)):.2f}")
print(f"  Win Rate: {metrics.get('win_rate', 0):.1f}%")
print(f"  Profit Factor: {metrics.get('profit_factor', 0):.2f}")

print(f"\nTrades:")
print(f"  Total Trades: {metrics.get('total_trades', 0)}")
print(f"  Winning Trades: {metrics.get('winning_trades', 0)}")
print(f"  Losing Trades: {metrics.get('losing_trades', 0)}")
print(f"  Average Win: ${metrics.get('avg_win', 0):.2f}")
print(f"  Average Loss: ${metrics.get('avg_loss', 0):.2f}")

print(f"\nRisk Metrics:")
print(f"  Max Drawdown: {metrics.get('max_drawdown', metrics.get('max_drawdown_pct', 0)):.2f}%")
print(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.2f}")
print(f"  Sortino Ratio: {metrics.get('sortino_ratio', 0):.2f}")
print("=" * 60)

## 3. Equity Curve

Visualize the equity progression over time.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Plot equity curve
ax.plot(equity_df['step'], equity_df['equity'], linewidth=2, color='#2E86AB')
ax.fill_between(equity_df['step'], equity_df['equity'], 
                equity_df['equity'].iloc[0], alpha=0.3, color='#2E86AB')

# Add horizontal line at starting capital
ax.axhline(y=equity_df['equity'].iloc[0], color='gray', 
           linestyle='--', linewidth=1, label='Starting Capital')

ax.set_xlabel('Time Step', fontsize=12)
ax.set_ylabel('Equity ($)', fontsize=12)
ax.set_title('Equity Curve', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print equity statistics
print(f"Starting Equity: ${equity_df['equity'].iloc[0]:,.2f}")
print(f"Final Equity: ${equity_df['equity'].iloc[-1]:,.2f}")
print(f"Total Return: {((equity_df['equity'].iloc[-1] / equity_df['equity'].iloc[0]) - 1) * 100:.2f}%")
print(f"Peak Equity: ${equity_df['equity'].max():,.2f}")

## 4. P&L Distribution

Analyze the distribution of profit and loss across all trades.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of P&L in USD
ax1 = axes[0]
winning_trades = trades_df[trades_df['pnl_usd'] > 0]
losing_trades = trades_df[trades_df['pnl_usd'] <= 0]

ax1.hist(winning_trades['pnl_usd'], bins=20, alpha=0.7, color='green', label='Wins')
ax1.hist(losing_trades['pnl_usd'], bins=20, alpha=0.7, color='red', label='Losses')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax1.set_xlabel('P&L (USD)', fontsize=11)
ax1.set_ylabel('Frequency', fontsize=11)
ax1.set_title('P&L Distribution (USD)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Histogram of P&L in percentage
ax2 = axes[1]
winning_pct = trades_df[trades_df['pnl_pct'] > 0]
losing_pct = trades_df[trades_df['pnl_pct'] <= 0]

ax2.hist(winning_pct['pnl_pct'], bins=20, alpha=0.7, color='green', label='Wins')
ax2.hist(losing_pct['pnl_pct'], bins=20, alpha=0.7, color='red', label='Losses')
ax2.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax2.set_xlabel('P&L (%)', fontsize=11)
ax2.set_ylabel('Frequency', fontsize=11)
ax2.set_title('P&L Distribution (%)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print distribution stats
print(f"P&L Statistics:")
print(f"  Mean P&L: ${trades_df['pnl_usd'].mean():.2f}")
print(f"  Median P&L: ${trades_df['pnl_usd'].median():.2f}")
print(f"  Std Dev: ${trades_df['pnl_usd'].std():.2f}")

## 5. Top Winning and Losing Trades

Identify and analyze the best and worst trades.

In [ ]:
# Top 10 winning trades
print("=" * 80)
print("TOP 10 WINNING TRADES")
print("=" * 80)
top_wins = trades_df.nlargest(10, 'pnl_usd')[['entry_time', 'exit_time', 'entry_price', 
                                                'exit_price', 'pnl_usd', 'pnl_pct']]
print(top_wins.to_string(index=False))
print()

# Top 10 losing trades
print("=" * 80)
print("TOP 10 LOSING TRADES")
print("=" * 80)
top_losses = trades_df.nsmallest(10, 'pnl_usd')[['entry_time', 'exit_time', 'entry_price', 
                                                   'exit_price', 'pnl_usd', 'pnl_pct']]
print(top_losses.to_string(index=False))

## 6. Trade Analysis by Time

In [ ]:
# Convert timestamps to datetime if they're not already
trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])

# Cumulative P&L over time
trades_df = trades_df.sort_values('exit_time')
trades_df['cumulative_pnl'] = trades_df['pnl_usd'].cumsum()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(trades_df['exit_time'], trades_df['cumulative_pnl'], linewidth=2, color='#A23B72')
ax.fill_between(trades_df['exit_time'], trades_df['cumulative_pnl'], 0, alpha=0.3, color='#A23B72')
ax.axhline(y=0, color='gray', linestyle='--', linewidth=1)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative P&L ($)', fontsize=12)
ax.set_title('Cumulative P&L Over Time', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Win/Loss Streak Analysis

In [ ]:
# Calculate streaks
trades_df['is_win'] = trades_df['pnl_usd'] > 0
trades_df['streak_id'] = (trades_df['is_win'] != trades_df['is_win'].shift()).cumsum()

# Find longest winning and losing streaks
streaks = trades_df.groupby(['streak_id', 'is_win']).size().reset_index(name='length')
win_streaks = streaks[streaks['is_win'] == True]
loss_streaks = streaks[streaks['is_win'] == False]

print("Streak Analysis:")
print(f"  Longest winning streak: {win_streaks['length'].max() if len(win_streaks) > 0 else 0} trades")
print(f"  Longest losing streak: {loss_streaks['length'].max() if len(loss_streaks) > 0 else 0} trades")
print(f"  Average winning streak: {win_streaks['length'].mean() if len(win_streaks) > 0 else 0:.2f} trades")
print(f"  Average losing streak: {loss_streaks['length'].mean() if len(loss_streaks) > 0 else 0:.2f} trades")

## 8. Drawdown Analysis

In [ ]:
# Calculate drawdown
equity_df['peak'] = equity_df['equity'].cummax()
equity_df['drawdown'] = (equity_df['equity'] - equity_df['peak']) / equity_df['peak'] * 100

fig, ax = plt.subplots(figsize=(14, 6))
ax.fill_between(equity_df['step'], equity_df['drawdown'], 0, 
                alpha=0.5, color='red', label='Drawdown')
ax.plot(equity_df['step'], equity_df['drawdown'], linewidth=2, color='darkred')
ax.set_xlabel('Time Step', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.set_title('Drawdown Over Time', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Max Drawdown: {equity_df['drawdown'].min():.2f}%")
print(f"Average Drawdown: {equity_df[equity_df['drawdown'] < 0]['drawdown'].mean():.2f}%")

## 9. Summary

Key takeaways from the backtest analysis.

In [ ]:
print("=" * 80)
print("BACKTEST ANALYSIS SUMMARY")
print("=" * 80)
print(f"\nStrategy Performance:")
print(f"  - Total trades executed: {len(trades_df)}")
print(f"  - Win rate: {metrics.get('win_rate', 0):.1f}%")
print(f"  - Total return: {((equity_df['equity'].iloc[-1] / equity_df['equity'].iloc[0]) - 1) * 100:.2f}%")
print(f"  - Profit factor: {metrics.get('profit_factor', 0):.2f}")
print(f"\nRisk Assessment:")
print(f"  - Maximum drawdown: {equity_df['drawdown'].min():.2f}%")
print(f"  - Sharpe ratio: {metrics.get('sharpe_ratio', 0):.2f}")
print(f"  - Best trade: ${trades_df['pnl_usd'].max():.2f}")
print(f"  - Worst trade: ${trades_df['pnl_usd'].min():.2f}")
print("\n" + "=" * 80)